In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import random
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import GradScaler, autocast

import cv2

import albumentations
from albumentations.pytorch import ToTensorV2

from sklearn.metrics import confusion_matrix, roc_auc_score, average_precision_score

from tqdm import tqdm
import argparse
import os, sys, yaml

sys.path.append('/workspace/siim-rsna-2021')
from src.logger import setup_logger, LOGGER
from src.meter import mAPMeter, AUCMeter, APMeter, AverageValueMeter
from src.utils import plot_sample_images
from src.segloss import SymmetricLovaszLoss


# import neptune.new as neptune
import wandb
import pydicom

import time
from contextlib import contextmanager

import timm

import warnings

target_columns = [
    "Negative for Pneumonia", "Typical Appearance", "Indeterminate Appearance", "Atypical Appearance", "is_none"
]


@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f'[{name}] done in {time.time() - t0:.0f} s')

In [2]:
from exp.exp414.train import Net as Net414
from exp.exp415.train import Net as Net415
from exp.exp416.train import Net as Net416
from exp.exp417.train import Net as Net417

from exp.exp418.train import Net as Net418
from exp.exp419.train import Net as Net419
from exp.exp420.train import Net as Net420

from exp.exp520.train import Net as Net520
from exp.exp551.train import Net as Net551
from exp.exp552.train import Net as Net552
from exp.exp553.train import Net as Net553

from exp.exp604.train import Net as Net604
from exp.exp605.train import Net as Net605
from exp.exp606.train import Net as Net606

In [3]:
class CustomDataset(Dataset):
    def __init__(self,
                 df,
                 image_size=512,
                 transform=None,
                 ):
        self.df = df
        self.image_size = image_size
        self.transform = transform
        self.cols = target_columns

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        row = self.df.iloc[index]
        images = cv2.imread(row.npy_path)

        # original image size
        original_h = images.shape[0]
        original_w = images.shape[1]
        images = cv2.resize(images, (512, 512))

        if self.transform is not None:
            aug = self.transform(image=images)
            images_only = aug['image'].astype(np.float32).transpose(2, 0, 1) / 255
        return {
            "image": torch.tensor(images_only, dtype=torch.float),
        }


In [4]:
def get_val_transforms(image_size=512):
    return albumentations.Compose([
        albumentations.Resize(image_size, image_size),
])

In [5]:
study_sub_list = [

    # prediction set, b6 mask
    [
        # backbone
        Net414("tf_efficientnetv2_m_in21k"),
        "tf_efficientnetv2_m_in21k",
        # img_size
        512,
        # weight list
        [
            "/workspace/output/exp414/model/cv0_weight_checkpoint_best.pth",
            "/workspace/output/exp414/model/cv1_weight_checkpoint_best.pth",
            "/workspace/output/exp414/model/cv2_weight_checkpoint_best.pth",
            "/workspace/output/exp414/model/cv3_weight_checkpoint_best.pth",
            "/workspace/output/exp414/model/cv4_weight_checkpoint_best.pth",
        ],
        "exp414_hflip"
    ],
    
    # prediction set, b6 mask
    [
        # backbone
        Net415("tf_efficientnetv2_m_in21k"),
        "tf_efficientnetv2_m_in21k",
        # img_size
        512,
        # weight list
        [
            "/workspace/output/exp415/model/cv0_weight_checkpoint_best.pth",
            "/workspace/output/exp415/model/cv1_weight_checkpoint_best.pth",
            "/workspace/output/exp415/model/cv2_weight_checkpoint_best.pth",
            "/workspace/output/exp415/model/cv3_weight_checkpoint_best.pth",
            "/workspace/output/exp415/model/cv4_weight_checkpoint_best.pth",
        ],
        "exp415_hflip"
    ],

    # prediction set, b7 mask
    [
        # backbone
        Net416("tf_efficientnetv2_m_in21k"),
        "tf_efficientnetv2_m_in21k",
        # img_size
        512,
        # weight list
        [
            "/workspace/output/exp416/model/cv0_weight_checkpoint_best.pth",
            "/workspace/output/exp416/model/cv1_weight_checkpoint_best.pth",
            "/workspace/output/exp416/model/cv2_weight_checkpoint_best.pth",
            "/workspace/output/exp416/model/cv3_weight_checkpoint_best.pth",
            "/workspace/output/exp416/model/cv4_weight_checkpoint_best.pth",
        ],
        "exp416_hflip"
    ],
    
    # prediction set, b6 mask
    [
        # backbone
        Net417("tf_efficientnetv2_m_in21k"),
        "tf_efficientnetv2_m_in21k",
        # img_size
        512,
        # weight list
        [
            "/workspace/output/exp417/model/cv0_weight_checkpoint_best.pth",
            "/workspace/output/exp417/model/cv1_weight_checkpoint_best.pth",
            "/workspace/output/exp417/model/cv2_weight_checkpoint_best.pth",
            "/workspace/output/exp417/model/cv3_weight_checkpoint_best.pth",
            "/workspace/output/exp417/model/cv4_weight_checkpoint_best.pth",
        ],
        "exp417_hflip"
    ],

    # ===========================================
    # Eff v2 L
    # ===========================================


    # prediction set, b7 map
    [
        # backbone
        Net419("tf_efficientnetv2_l_in21k"),
        "tf_efficientnetv2_l_in21k",
        # img_size
        512,
        # weight list
        [
            "/workspace/output/exp419/model/cv0_weight_checkpoint_best.pth",
            "/workspace/output/exp419/model/cv1_weight_checkpoint_best.pth",
            "/workspace/output/exp419/model/cv2_weight_checkpoint_best.pth",
            "/workspace/output/exp419/model/cv3_weight_checkpoint_best.pth",
            "/workspace/output/exp419/model/cv4_weight_checkpoint_best.pth",
        ],
        "exp419_hflip"
    ],

    # prediction set, b6, b7
    [
        # backbone
        Net420("tf_efficientnetv2_l_in21k"),
        "tf_efficientnetv2_l_in21k",
        # img_size
        512,
        # weight list
        [
            "/workspace/output/exp420/model/cv0_weight_checkpoint_best.pth",
            "/workspace/output/exp420/model/cv1_weight_checkpoint_best.pth",
            "/workspace/output/exp420/model/cv2_weight_checkpoint_best.pth",
            "/workspace/output/exp420/model/cv3_weight_checkpoint_best.pth",
            "/workspace/output/exp420/model/cv4_weight_checkpoint_best.pth",
        ],
        "exp420_hflip"
    ],
    
    # prediction set, b6, b7
    [
        # backbone
        Net420("tf_efficientnetv2_l_in21k"),
        "tf_efficientnetv2_l_in21k",
        # img_size
        512,
        # weight list
        [
            "/workspace/output/exp520/model/cv0_weight_checkpoint_best.pth",
            "/workspace/output/exp520/model/cv1_weight_checkpoint_best.pth",
            "/workspace/output/exp520/model/cv2_weight_checkpoint_best.pth",
            "/workspace/output/exp520/model/cv3_weight_checkpoint_best.pth",
            "/workspace/output/exp520/model/cv4_weight_checkpoint_best.pth",
        ],
        "exp520_hflip"
    ],
    
    # prediction set, b6
    [
        # backbone
        Net418("tf_efficientnetv2_l_in21k"),
        "tf_efficientnetv2_l_in21k",
        # img_size
        512,
        # weight list
        [
            "/workspace/output/exp551/model/cv0_weight_checkpoint_best.pth",
            "/workspace/output/exp551/model/cv1_weight_checkpoint_best.pth",
            "/workspace/output/exp551/model/cv2_weight_checkpoint_best.pth",
            "/workspace/output/exp551/model/cv3_weight_checkpoint_best.pth",
            "/workspace/output/exp551/model/cv4_weight_checkpoint_best.pth",
        ],
        "exp551_hflip"
    ],
    
    # prediction set, b6
    [
        # backbone
        Net552("tf_efficientnetv2_l_in21k"),
        "tf_efficientnetv2_l_in21k",
        # img_size
        512,
        # weight list
        [
            "/workspace/output/exp552/model/cv0_weight_checkpoint_best.pth",
            "/workspace/output/exp552/model/cv1_weight_checkpoint_best.pth",
            "/workspace/output/exp552/model/cv2_weight_checkpoint_best.pth",
            "/workspace/output/exp552/model/cv3_weight_checkpoint_best.pth",
            "/workspace/output/exp552/model/cv4_weight_checkpoint_best.pth",
        ],
        "exp552_hflip"
    ],
    
    # prediction set, b6
    [
        # backbone
        Net553("tf_efficientnetv2_l_in21k"),
        "tf_efficientnetv2_l_in21k",
        # img_size
        512,
        # weight list
        [
            "/workspace/output/exp553/model/cv0_weight_checkpoint_best.pth",
            "/workspace/output/exp553/model/cv1_weight_checkpoint_best.pth",
            "/workspace/output/exp553/model/cv2_weight_checkpoint_best.pth",
            "/workspace/output/exp553/model/cv3_weight_checkpoint_best.pth",
            "/workspace/output/exp553/model/cv4_weight_checkpoint_best.pth",
        ],
        "exp553_hflip"
    ],

    # ===========================================
    # Swin transformer
    # ===========================================

    # prediction set
    [
        # backbone
        Net604("swin_base_patch4_window12_384"),
        "swin_base_patch4_window12_384",
        # img_size
        384,
        # weight list
        [
            "/workspace/output/exp604/model/cv0_weight_checkpoint_best.pth",
            "/workspace/output/exp604/model/cv1_weight_checkpoint_best.pth",
            "/workspace/output/exp604/model/cv2_weight_checkpoint_best.pth",
            "/workspace/output/exp604/model/cv3_weight_checkpoint_best.pth",
            "/workspace/output/exp604/model/cv4_weight_checkpoint_best.pth",
        ],
        "exp604_hflip"
    ],
    
    # prediction set
    [
        # backbone
        Net605("swin_base_patch4_window12_384"),
        "swin_base_patch4_window12_384",
        # img_size
        384,
        # weight list
        [
            "/workspace/output/exp605/model/cv0_weight_checkpoint_best.pth",
            "/workspace/output/exp605/model/cv1_weight_checkpoint_best.pth",
            "/workspace/output/exp605/model/cv2_weight_checkpoint_best.pth",
            "/workspace/output/exp605/model/cv3_weight_checkpoint_best.pth",
            "/workspace/output/exp605/model/cv4_weight_checkpoint_best.pth",
        ],
        "exp605_hflip"
    ],
    
    # prediction set
    [
        # backbone
        Net606("swin_base_patch4_window12_384"),
        "swin_base_patch4_window12_384",
        # img_size
        384,
        # weight list
        [
            "/workspace/output/exp606/model/cv0_weight_checkpoint_best.pth",
            "/workspace/output/exp606/model/cv1_weight_checkpoint_best.pth",
            "/workspace/output/exp606/model/cv2_weight_checkpoint_best.pth",
            "/workspace/output/exp606/model/cv3_weight_checkpoint_best.pth",
            "/workspace/output/exp606/model/cv4_weight_checkpoint_best.pth",
        ],
        "exp606_hflip"
    ],
    
    

]

In [6]:
device = "cuda:1"
hflip = True

In [7]:
df_original = pd.read_csv("/workspace/data/df_train_study_level_npy640_3_w_bbox.csv")
df = df_original.groupby('image_id').first().reset_index()

In [9]:
from copy import deepcopy

# key: image size, value: model
model_dict = {}

image_size_list = []

for model_set in tqdm(study_sub_list):
    # 画像サイズをkey, modelのリストをvalueにする
    # keyがまだない場合はからのリストを登録
    model_dict.setdefault(model_set[2], [])
    model_list = []
    exp_name = model_set[-1]
    for cv, ckpt in tqdm(enumerate(model_set[3])):
        model = model_set[0].to(device)
        weight = torch.load(ckpt, map_location=device)
        model.load_state_dict(weight["state_dict"])
        model.eval()
        
        df_val = df[df.cv == cv].reset_index(drop=True)
        
        dataset = CustomDataset(df=df_val, transform=get_val_transforms(model_set[2]))
        test_loader = DataLoader(
            dataset,
            shuffle=False,
            batch_size=32,
            num_workers=0,
            pin_memory=True,
        )
        
        pred_list1 = []
        for i, image in enumerate(test_loader):
            pred_list2 = []
            pred_mask2 = []
            image = image["image"].to(device)
            with torch.no_grad():
                preds, *_ = model(image)
                preds = preds.cpu().detach()
                pred_list2.append(preds.sigmoid())
                
                if hflip:
                    preds, *_ = model(image.flip(-1))
                    pred_list2.append(preds.cpu().detach().sigmoid())

                # average prediction
                pred_list1.append(torch.stack(pred_list2, 0).mean(0))

        preds = torch.cat(pred_list1).numpy()
        print(f"preds.shape: {preds.shape}")
        np.save(f'/workspace/output/oof/{exp_name}_cv{cv}.npy', preds)

  0%|          | 0/13 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [01:57, 117.82s/it]

preds.shape: (1223, 5)



2it [03:53, 117.31s/it]

preds.shape: (1220, 5)



3it [05:51, 117.28s/it]

...

preds.shape: (1228, 5)



5it [10:46, 129.31s/it]
 92%|█████████▏| 12/13 [2:11:21<11:06, 666.80s/it]
0it [00:00, ?it/s]

preds.shape: (1225, 5)


0it [00:00, ?it/s]
 92%|█████████▏| 12/13 [2:11:22<10:56, 656.84s/it]


RuntimeError: Error(s) in loading state_dict for Net:
	Unexpected key(s) in state_dict: "mask1.0.weight", "mask1.0.bias", "mask1.2.weight", "mask1.2.bias", "mask1.4.weight", "mask1.4.bias", "mask2.0.weight", "mask2.0.bias", "mask2.2.weight", "mask2.2.bias", "mask2.4.weight", "mask2.4.bias". 